In [2]:
pip install pandas

     |████████████████████████████████| 11.8 MB 2.6 MB/s eta 0:00:011    |█████████████▋                  | 5.0 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 19.8 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 341 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 502 kB 18.2 MB/s eta 0:00:01
You should consider upgrading via the '/Users/jonah_1/.pyenv/versions/3.9.7/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [6]:
pd.read_csv('./Users/jonah_1/Downloads/interval_and_weather_merged')

,Unnamed: 0,endtime,starttime,interval,departing_insect,arriving_insect,sheet_title,timestamp_pd,timestamp_pd_rounded,same_insect,...,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,0,0 days 11:41:43,0 days 11:44:13,150000000000,Insecta,Insecta,07-28-2022 i10 Zinnia,2022-07-28 11:41:43,2022-07-28 12:00:00,1,...,23.0,100.0,0.0,NaN,270.0,7.2,NaN,1014.0,NaN,3.0
1,1,0 days 11:44:43,0 days 12:04:13,1170000000000,Insecta,Syrphinae,07-28-2022 i10 Zinnia,2022-07-28 11:44:43,2022-07-28 12:00:00,0,...,23.0,100.0,0.0,NaN,270.0,7.2,NaN,1014.0,NaN,3.0
2,2,0 days 13:43:07,0 days 13:54:07,660000000000,Halictus ligatus/poeyi,Syrphidae,07-28-2022 i4 Black-eyed Susan,2022-07-28 13:43:07,2022-07-28 14:00:00,0,...,25.0,79.0,0.0,NaN,271.0,8.3,NaN,1013.0,NaN,3.0
3,3,0 days 12:57:16,0 days 12:59:20,124000000000,Augochlorini,Apis mellifera,08-04-2022 i8 Cup Plant,2022-08-04 12:57:16,2022-08-04 13:00:00,1,...,25.0,84.0,0.0,NaN,220.0,7.0,NaN,1019.0,NaN,1.0
4,4,0 days 13:00:22,0 days 13:00:53,31000000000,Apis mellifera,Augochlorini,08-04-2022 i8 Cup Plant,2022-08-04 13:00:22,2022-08-04 13:00:00,1,...,25.0,84.0,0.0,NaN,220.0,7.0,NaN,1019.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,271,0 days 00:00:41,0 days 00:03:07,146000000000,B. impatiens (F),B. impatiens (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:19:29,2022-10-14 13:00:00,1,...,12.0,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0
294,272,0 days 00:03:14,0 days 00:09:39,385000000000,B. impatiens (F),B. impatiens (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:21:59,2022-10-14 13:00:00,1,...,12.0,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0
295,273,0 days 00:09:57,0 days 00:10:14,17000000000,B. impatiens (F),Lasioglossum (Dialictus) sp. (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:28:31,2022-10-14 13:00:00,0,...,12.0,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0
296,274,0 days 00:10:36,0 days 00:10:39,3000000000,Lasioglossum (Dialictus) sp. (F),L. (Dialictus) sp. (F),(pol.) +Tithonia rotundifolia 10.14.22,2022-10-14 13:29:06,2022-10-14 13:00:00,1,...,12.0,100.0,0.0,NaN,210.0,6.0,NaN,1016.0,NaN,3.0
